In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("stats (26) - Copy.csv")

df.head()



,Name,player_id,year,age,ab,pa,H,1B,2B,3B,HR,K,BB,RBI,CS,SB,GIDP
0,"Pujols, Albert",405395,2020,40,152,163,34,20,8,0,6,25,9,25,0,0,4
1,"Cabrera, Miguel",408234,2020,37,204,231,51,37,4,0,10,51,24,35,0,1,3
2,"Choo, Shin-Soo",425783,2020,37,110,127,26,18,3,0,5,33,13,15,2,6,1
3,"Molina, Yadier",425877,2020,37,145,156,38,32,2,0,4,21,6,16,0,0,7
4,"Canó, Robinson",429664,2020,37,171,182,54,35,9,0,10,24,9,30,0,0,7


In [4]:
df.info()
df[['age', 'year']].describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619 entries, 0 to 2618
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       2619 non-null   object
 1   player_id  2619 non-null   int64 
 2   year       2619 non-null   int64 
 3   age        2619 non-null   int64 
 4   ab         2619 non-null   int64 
 5   pa         2619 non-null   int64 
 6   H          2619 non-null   int64 
 7   1B         2619 non-null   int64 
 8   2B         2619 non-null   int64 
 9   3B         2619 non-null   int64 
 10  HR         2619 non-null   int64 
 11  K          2619 non-null   int64 
 12  BB         2619 non-null   int64 
 13  RBI        2619 non-null   int64 
 14  CS         2619 non-null   int64 
 15  SB         2619 non-null   int64 
 16  GIDP       2619 non-null   int64 
dtypes: int64(16), object(1)
memory usage: 348.0+ KB


,age,year
count,2619.000000,2619.000000
mean,28.029019,2022.638030
std,3.704714,1.642965
min,20.000000,2020.000000
25%,25.000000,2021.000000
50%,28.000000,2023.000000
75%,30.000000,2024.000000
max,42.000000,2025.000000


In [7]:
counting_stats = [
    'pa',
    'ab',
    'H',
    'HR',
    'RBI',
    'BB',
    'K'
]

age_avg = (
    df
    .groupby('age')[counting_stats]
    .mean()
    .reset_index()
    .sort_values('age')
)

age_avg.head(10)


,age,pa,ab,H,HR,RBI,BB,K
0,20,253.833333,234.500000,59.666667,7.500000,31.666667,16.666667,54.000000
1,21,304.800000,276.628571,70.514286,10.257143,35.485714,23.428571,69.800000
2,22,340.853333,306.066667,76.466667,11.026667,38.666667,28.040000,80.200000
3,23,323.510345,290.800000,71.393103,9.793103,35.296552,26.413793,75.558621
4,24,342.780612,308.918367,76.760204,10.505102,38.755102,26.892857,80.352041
5,25,336.124031,301.806202,74.662791,10.275194,38.244186,27.333333,76.624031
6,26,341.669014,306.320423,76.559859,10.320423,38.633803,28.295775,75.359155
7,27,363.011029,324.750000,81.253676,11.338235,41.272059,30.702206,79.242647
8,28,353.496183,314.522901,77.061069,11.286260,41.576336,31.309160,78.576336
9,29,362.683258,323.642534,80.484163,12.167421,42.832579,31.751131,80.665158


In [9]:
age_deltas = age_avg.copy()

for stat in counting_stats:
    age_deltas[f'{stat}_delta'] = age_deltas[stat].diff()

age_deltas = age_deltas.dropna()
age_deltas = age_deltas[['age'] + [f'{s}_delta' for s in counting_stats]]


In [10]:
age_avg_smooth = age_avg.copy()

for stat in counting_stats:
    age_avg_smooth[f'{stat}_smooth'] = (
        age_avg_smooth[stat]
        .rolling(window=3, center=True, min_periods=1)
        .mean()
    )


In [12]:
df = df.merge(age_deltas, on='age', how='left')


In [15]:
df = df.sort_values(['player_id', 'year'])

df['HR_next'] = df.groupby('player_id')['HR'].shift(-1)

reg_df = df.dropna(subset=['HR_next', 'HR', 'age'])
from sklearn.linear_model import LinearRegression

X = reg_df[['HR', 'age']]
y = reg_df['HR_next']

model = LinearRegression()
model.fit(X, y)

model.coef_, model.intercept_


(array([ 0.59847914, -0.32013874]), np.float64(14.818986355176438))

Projection models

In [18]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# ---------------------------
# 1) Load
# ---------------------------
df = pd.read_csv("stats (26) - Copy.csv")

# Ensure correct dtypes
df['year'] = df['year'].astype(int)
df['age'] = df['age'].astype(int)

# Sort for time-series operations
df = df.sort_values(['player_id', 'year']).reset_index(drop=True)

# ---------------------------
# 2) Define counting stats (everything in file except identifiers)
# ---------------------------
id_cols = ['Name', 'player_id', 'year', 'age']
counting_stats = [c for c in df.columns if c not in id_cols]

print("Counting stats:", counting_stats)

# ---------------------------
# 3) Build age averages + age deltas (league-wide aging curve)
#    Delta at age X = mean(age X) - mean(age X-1)
#    For a player age A, next-year delta uses age (A+1)
# ---------------------------
age_avg = (
    df.groupby('age')[counting_stats]
      .mean()
      .reset_index()
      .sort_values('age')
)

age_deltas = age_avg.copy()
for s in counting_stats:
    age_deltas[f'{s}_age_delta'] = age_deltas[s].diff()

age_deltas = age_deltas[['age'] + [f'{s}_age_delta' for s in counting_stats]].dropna()

# Merge deltas onto each player-season row by their CURRENT age
# We'll also build "next_age_delta" features by merging on (age + 1) later.
df = df.merge(age_deltas, on='age', how='left')

# ---------------------------
# 4) Create next-year targets per stat
# ---------------------------
for s in counting_stats:
    df[f'{s}_next'] = df.groupby('player_id')[s].shift(-1)

# Keep only rows where we actually observe next year (for training)
train_df = df.dropna(subset=[f'{s}_next' for s in counting_stats]).copy()

# ---------------------------
# 5) Helper: find players by name
# ---------------------------
def find_players(name_query: str, max_results: int = 15):
    q = name_query.strip().lower()
    matches = (df[['Name', 'player_id']]
               .drop_duplicates()
               .assign(Name_lower=lambda x: x['Name'].str.lower())
               .loc[lambda x: x['Name_lower'].str.contains(q, na=False)]
               .drop(columns=['Name_lower'])
               .head(max_results))
    return matches

# ---------------------------
# 6) Build per-stat regression models
#    Features:
#      - current stat level (s)
#      - age 
#      - league aging delta for that stat at *next age* (age+1)
#      - playing-time control (pa), because counting stats scale with it
#    Target:
#      - s_next
# ---------------------------
# Build a "next age delta" table: keyed by next year's age
next_age_deltas = age_deltas.rename(columns={'age': 'next_age'}).copy()

def train_models(stats=counting_stats):
    models = {}
    metrics = []

    for s in stats:
        # Create a working frame for this stat
        tmp = train_df[['player_id', 'year', 'age', 'pa', s, f'{s}_next']].copy()

        # attach delta for next age (A+1)
        tmp['next_age'] = tmp['age'] + 1
        tmp = tmp.merge(
            next_age_deltas[['next_age', f'{s}_age_delta']],
            on='next_age',
            how='left'
        )

        # some ages at the extremes may not have deltas; drop those
        tmp = tmp.dropna(subset=[f'{s}_age_delta'])

        X = tmp[[s, 'age', 'pa', f'{s}_age_delta']].values
        y = tmp[f'{s}_next'].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        models[s] = model
        metrics.append({'stat': s, 'MAE': mae, 'R2': r2, 'n_train': len(X_train), 'n_test': len(X_test)})

    metrics_df = pd.DataFrame(metrics).sort_values('R2', ascending=False)
    return models, metrics_df

models, model_metrics = train_models()
model_metrics


Counting stats: ['ab', 'pa', 'H', '1B', '2B', '3B', 'HR', 'K', 'BB', 'RBI', 'CS', 'SB', 'GIDP']


,stat,MAE,R2,n_train,n_test
11,SB,4.717306,0.428162,1413,354
8,BB,13.667201,0.382424,1413,354
10,CS,1.460601,0.362938,1413,354
6,HR,6.681410,0.303574,1413,354
2,H,31.587592,0.269214,1413,354
3,1B,20.157321,0.265628,1413,354
7,K,29.710022,0.218091,1413,354
9,RBI,19.585267,0.215209,1413,354
1,pa,128.513900,0.207548,1413,354
0,ab,112.945202,0.205736,1413,354


projections

In [27]:
# ============================================================
# ONE-CELL PLAYER SEARCH UI + HISTORY + NEXT-YEAR PROJECTIONS
# (Clean version: fixes Series->float issue AND sklearn feature-name warnings)
# ============================================================

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------------------------
# Helpers
# ---------------------------
def to_scalar(x):
    """Safely convert a value (or a Series from duplicate cols) to float or NaN."""
    if isinstance(x, pd.Series):
        x = x.iloc[0]
    x = pd.to_numeric(x, errors='coerce')
    return float(x) if pd.notna(x) else np.nan

# ---------------------------
# 1) Load + clean
# ---------------------------
df = pd.read_csv("stats (26) - Copy.csv").copy()

# Remove duplicate column names
df = df.loc[:, ~df.columns.duplicated()].copy()

# Required columns
required_cols = {'Name', 'player_id', 'year', 'age', 'pa'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

# Coerce types
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['age']  = pd.to_numeric(df['age'], errors='coerce')
df['pa']   = pd.to_numeric(df['pa'], errors='coerce')
df['player_id'] = pd.to_numeric(df['player_id'], errors='coerce')

# Drop rows missing essentials
df = df.dropna(subset=['player_id', 'year', 'age']).copy()

# Convert to ints where appropriate
df['year'] = df['year'].astype(int)
df['age']  = df['age'].astype(int)
df['player_id'] = df['player_id'].astype(int)

# ---------------------------
# 2) Counting stats (everything except identifiers)
# ---------------------------
id_cols = ['Name', 'player_id', 'year', 'age']
counting_stats = [c for c in df.columns if c not in id_cols]

# Coerce ALL counting stats to numeric (bad values -> NaN)
for c in counting_stats:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Sort for time series operations
df = df.sort_values(['player_id', 'year']).reset_index(drop=True)

# ---------------------------
# 3) Build league-wide age averages + age deltas
# ---------------------------
age_avg = (
    df.groupby('age')[counting_stats]
      .mean()
      .reset_index()
      .sort_values('age')
)

age_deltas = age_avg.copy()
for s in counting_stats:
    age_deltas[f'{s}_age_delta'] = age_deltas[s].diff()

age_deltas = age_deltas[['age'] + [f'{s}_age_delta' for s in counting_stats]].dropna()
next_age_deltas = age_deltas.rename(columns={'age': 'next_age'}).copy()

# ---------------------------
# 4) Create next-year targets
# ---------------------------
for s in counting_stats:
    df[f'{s}_next'] = df.groupby('player_id')[s].shift(-1)

# Training data = rows where we observe next year
train_df = df.dropna(subset=[f'{s}_next' for s in counting_stats]).copy()

# ---------------------------
# 5) Train per-stat regression models
# Features: current stat, age, pa, league delta at next age
# Target: next year stat
# ---------------------------
models = {}
metrics_rows = []

for s in counting_stats:
    # Build training frame for this stat
    tmp = train_df[['player_id', 'year', 'age', 'pa', s, f'{s}_next']].copy()
    tmp['next_age'] = tmp['age'] + 1

    tmp = tmp.merge(
        next_age_deltas[['next_age', f'{s}_age_delta']],
        on='next_age',
        how='left'
    ).dropna(subset=[f'{s}_age_delta'])

    # Skip if too few rows
    if len(tmp) < 50:
        continue

    feature_cols = [s, 'age', 'pa', f'{s}_age_delta']
    X = tmp[feature_cols]
    y = tmp[f'{s}_next']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = LinearRegression()
    model.fit(X_train, y_train)
    models[s] = (model, feature_cols)  # store feature columns too

    y_pred = model.predict(X_test)
    metrics_rows.append({
        'stat': s,
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred),
        'n_train': len(X_train),
        'n_test': len(X_test),
    })

model_metrics = pd.DataFrame(metrics_rows).sort_values('R2', ascending=False)

# ---------------------------
# 6) Player lookup + history + projection
# ---------------------------
def get_matches(name_text: str, limit=25):
    q = (name_text or "").strip().lower()
    if not q:
        return []
    matches = (
        df[['Name','player_id']]
        .drop_duplicates()
        .assign(_n=lambda x: x['Name'].str.lower())
        .loc[lambda x: x['_n'].str.contains(q, na=False)]
        .drop(columns=['_n'])
        .head(limit)
    )
    return [(f"{r['Name']}  (id={int(r['player_id'])})", int(r['player_id'])) for _, r in matches.iterrows()]

def player_history(pid: int):
    cols = ['year', 'age', 'pa'] + [c for c in counting_stats if c != 'pa']
    return df.loc[df['player_id'] == pid, ['Name'] + cols].sort_values('year')

def project_next_year(pid: int):
    hist = player_history(pid)
    if hist.empty:
        return hist, pd.DataFrame(), pd.DataFrame(), {}

    last = hist.sort_values('year').iloc[-1]
    age_now = int(last['age'])
    age_next = age_now + 1

    # Last year snapshot (single-row table)
    last_year_stats = {'year': int(last['year']), 'age': age_now, 'pa': to_scalar(last['pa'])}
    for s in [c for c in counting_stats if c not in ['pa']]:
        last_year_stats[s] = to_scalar(last[s])
    last_year_df = pd.DataFrame([last_year_stats])

    # Projections
    rows = []
    for s in [c for c in counting_stats if c not in ['pa']]:
        current = to_scalar(last[s])
        pa_val  = to_scalar(last['pa'])

        delta_series = age_deltas.loc[age_deltas['age'] == age_next, f'{s}_age_delta']
        age_delta = to_scalar(delta_series) if len(delta_series) else np.nan

        age_only_proj = current + age_delta if np.isfinite(age_delta) else np.nan

        reg_proj = np.nan
        if s in models and np.isfinite(age_delta) and np.isfinite(pa_val) and np.isfinite(current):
            model, feature_cols = models[s]

            # IMPORTANT FIX: use a 1-row DataFrame with correct feature names
            X_pred = pd.DataFrame(
                [[current, age_now, pa_val, age_delta]],
                columns=feature_cols
            )
            reg_proj = float(model.predict(X_pred)[0])

        blend = np.nan
        if np.isfinite(age_only_proj) and np.isfinite(reg_proj):
            blend = 0.5 * age_only_proj + 0.5 * reg_proj

        rows.append({
            'stat': s,
            'current': current,
            'age_delta(next_age)': age_delta,
            'age_only_proj': age_only_proj,
            'regression_proj': reg_proj,
            'blend_50_50': blend
        })

    proj = pd.DataFrame(rows).sort_values('stat')
    meta = {
        'Name': last['Name'],
        'last_year': int(last['year']),
        'age_now': age_now,
        'age_next': age_next
    }
    return hist, last_year_df, proj, meta

# ---------------------------
# 7) Notebook "Search Bar" UI
# ---------------------------
name_box = widgets.Text(
    value="",
    placeholder="Type full or partial name (e.g., 'Mookie Betts')",
    description="Search:",
    layout=widgets.Layout(width='650px')
)

results_dd = widgets.Dropdown(
    options=[],
    description="Matches:",
    layout=widgets.Layout(width='650px')
)

out = widgets.Output()

def refresh_matches(change=None):
    opts = get_matches(name_box.value)
    results_dd.options = opts
    if opts:
        results_dd.value = opts[0][1]

def render_player(change=None):
    with out:
        clear_output(wait=True)

        if results_dd.value is None:
            print("No player selected.")
            return

        hist, last_year_df, proj, meta = project_next_year(int(results_dd.value))

        print(meta)
        print("\n--- Last Year Snapshot ---")
        display(last_year_df)

        print("\n--- Full Prior Years ---")
        display(hist)

        print("\n--- Next Year Projections ---")
        display(proj)

        print("\n--- Model quality by stat (higher R2 is better) ---")
        display(model_metrics)

name_box.observe(refresh_matches, names='value')
results_dd.observe(render_player, names='value')

display(name_box, results_dd, out)

print("✅ Ready. Type a name above → pick a match → stats + projections display below.")


Text(value='', description='Search:', layout=Layout(width='650px'), placeholder="Type full or partial name (e.…

Dropdown(description='Matches:', layout=Layout(width='650px'), options=(), value=None)

Output()

✅ Ready. Type a name above → pick a match → stats + projections display below.


In [32]:
# ============================================================
# ONE-CELL PLAYER SEARCH UI + HISTORY + NEXT-YEAR PROJECTIONS
# + Adds AB-based RATE STATS (BB/AB, HR/AB, etc.)
# + FIXES the recurring sklearn error:
#   "Feature names must be in the same order as they were in fit."
#
# Root cause: even if we *think* we're matching feature_cols, sklearn is strict and
# sometimes the stored order differs. The bulletproof solution is to use:
#   model.feature_names_in_
# to build the prediction row in the exact order sklearn expects.
# ============================================================

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------------------------
# Helpers
# ---------------------------
def to_scalar(x):
    if isinstance(x, pd.Series):
        x = x.iloc[0]
    x = pd.to_numeric(x, errors='coerce')
    return float(x) if pd.notna(x) else np.nan

def safe_div(num, den):
    num = pd.to_numeric(num, errors='coerce')
    den = pd.to_numeric(den, errors='coerce')
    return np.where((den > 0) & np.isfinite(den) & np.isfinite(num), num / den, np.nan)

def build_X_pred(model, stat_name, current, age_now, pa_val, age_delta):
    """
    Build a 1-row DataFrame with columns EXACTLY in the order used during fit.
    Uses model.feature_names_in_ (sklearn's stored feature order).
    """
    cols = list(model.feature_names_in_)  # exact order used in fit
    row = {}
    for c in cols:
        if c == stat_name:
            row[c] = current
        elif c == 'age':
            row[c] = age_now
        elif c == 'pa':
            row[c] = pa_val
        elif c == f'{stat_name}_age_delta':
            row[c] = age_delta
        else:
            # Shouldn't happen, but safe default
            row[c] = np.nan
    X_pred = pd.DataFrame([row], columns=cols)
    return X_pred

# ---------------------------
# 1) Load + clean
# ---------------------------
df = pd.read_csv("stats (26) - Copy.csv").copy()
df = df.loc[:, ~df.columns.duplicated()].copy()

required_cols = {'Name', 'player_id', 'year', 'age', 'pa', 'ab'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

for col in ['year','age','pa','ab','player_id']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=['player_id','year','age']).copy()
df['year'] = df['year'].astype(int)
df['age']  = df['age'].astype(int)
df['player_id'] = df['player_id'].astype(int)

# ---------------------------
# 2) Counting stats
# ---------------------------
id_cols = ['Name', 'player_id', 'year', 'age']
counting_stats = [c for c in df.columns if c not in id_cols]

for c in counting_stats:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# ---------------------------
# 3) AB-based rate stats
# ---------------------------
rate_stats = []

if 'BB' in df.columns:
    df['BB_per_AB'] = safe_div(df['BB'], df['ab'])
    rate_stats.append('BB_per_AB')

if 'HR' in df.columns:
    df['HR_per_AB'] = safe_div(df['HR'], df['ab'])
    rate_stats.append('HR_per_AB')

# Optional extras (comment out if you want fewer)
if 'H' in df.columns:
    df['H_per_AB'] = safe_div(df['H'], df['ab'])
    rate_stats.append('H_per_AB')

if 'K' in df.columns:
    df['K_per_AB'] = safe_div(df['K'], df['ab'])
    rate_stats.append('K_per_AB')

for c in rate_stats:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.sort_values(['player_id','year']).reset_index(drop=True)

# ---------------------------
# 4) Age curves/deltas
# ---------------------------
model_stats = counting_stats + rate_stats

age_avg = (
    df.groupby('age')[model_stats]
      .mean()
      .reset_index()
      .sort_values('age')
)

age_deltas = age_avg.copy()
for s in model_stats:
    age_deltas[f'{s}_age_delta'] = age_deltas[s].diff()

age_deltas = age_deltas[['age'] + [f'{s}_age_delta' for s in model_stats]].dropna()
next_age_deltas = age_deltas.rename(columns={'age':'next_age'}).copy()

# ---------------------------
# 5) Next-year targets
# ---------------------------
for s in model_stats:
    df[f'{s}_next'] = df.groupby('player_id')[s].shift(-1)

train_df = df.dropna(subset=[f'{s}_next' for s in model_stats]).copy()

# ---------------------------
# 6) Train per-stat models (store model only; prediction uses model.feature_names_in_)
# ---------------------------
models = {}
metrics_rows = []

for s in model_stats:
    tmp = train_df[['player_id','year','age','pa',s,f'{s}_next']].copy()
    tmp['next_age'] = tmp['age'] + 1

    tmp = tmp.merge(
        next_age_deltas[['next_age', f'{s}_age_delta']],
        on='next_age',
        how='left'
    ).dropna(subset=[f'{s}_age_delta'])

    if len(tmp) < 50:
        continue

    feature_cols = [s, 'age', 'pa', f'{s}_age_delta']
    X = tmp[feature_cols]   # DataFrame with names => sklearn stores feature_names_in_
    y = tmp[f'{s}_next']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    models[s] = model

    y_pred = model.predict(X_test)
    metrics_rows.append({
        'stat': s,
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred),
        'n_train': len(X_train),
        'n_test': len(X_test),
    })

model_metrics = pd.DataFrame(metrics_rows).sort_values('R2', ascending=False)

# ---------------------------
# 7) Lookup + history + projections
# ---------------------------
def get_matches(name_text: str, limit=25):
    q = (name_text or "").strip().lower()
    if not q:
        return []
    matches = (
        df[['Name','player_id']]
        .drop_duplicates()
        .assign(_n=lambda x: x['Name'].str.lower())
        .loc[lambda x: x['_n'].str.contains(q, na=False)]
        .drop(columns=['_n'])
        .head(limit)
    )
    return [(f"{r['Name']}  (id={int(r['player_id'])})", int(r['player_id'])) for _, r in matches.iterrows()]

def player_history(pid: int):
    cols = ['year','age','pa','ab'] + [c for c in counting_stats if c not in ['pa','ab']] + rate_stats
    cols = [c for c in cols if c in df.columns]
    return df.loc[df['player_id']==pid, ['Name'] + cols].sort_values('year')

def project_next_year(pid: int):
    hist = player_history(pid)
    if hist.empty:
        return hist, pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), {}

    last = hist.sort_values('year').iloc[-1]
    age_now = int(last['age'])
    age_next = age_now + 1

    # Last year snapshot
    last_year = {'year': int(last['year']), 'age': age_now}
    for col in ['pa','ab'] + [c for c in counting_stats if c not in ['pa','ab']] + rate_stats:
        if col in last.index:
            last_year[col] = to_scalar(last[col])
    last_year_df = pd.DataFrame([last_year])

    # Projections table
    proj_rows = []
    for s in model_stats:
        if s not in last.index:
            continue

        current = to_scalar(last[s])
        pa_val = to_scalar(last['pa']) if 'pa' in last.index else np.nan

        delta_series = age_deltas.loc[age_deltas['age']==age_next, f'{s}_age_delta']
        age_delta = to_scalar(delta_series) if len(delta_series) else np.nan

        age_only_proj = current + age_delta if np.isfinite(age_delta) else np.nan

        reg_proj = np.nan
        if s in models and np.isfinite(age_delta) and np.isfinite(pa_val) and np.isfinite(current):
            model = models[s]
            X_pred = build_X_pred(model, s, current, age_now, pa_val, age_delta)  # bulletproof order
            reg_proj = float(model.predict(X_pred)[0])

        blend = np.nan
        if np.isfinite(age_only_proj) and np.isfinite(reg_proj):
            blend = 0.5 * age_only_proj + 0.5 * reg_proj

        proj_rows.append({
            'stat': s,
            'current': current,
            'age_delta(next_age)': age_delta,
            'age_only_proj': age_only_proj,
            'regression_proj': reg_proj,
            'blend_50_50': blend
        })

    proj = pd.DataFrame(proj_rows).sort_values('stat')

    # Implied counts from predicted AB * predicted rate (blend)
    implied = []
    ab_next_pred = np.nan
    if 'ab' in proj['stat'].values:
        ab_next_pred = float(proj.loc[proj['stat']=='ab', 'blend_50_50'].iloc[0])

    if np.isfinite(ab_next_pred):
        for r in ['BB_per_AB', 'HR_per_AB', 'H_per_AB', 'K_per_AB']:
            if r in proj['stat'].values:
                r_pred = float(proj.loc[proj['stat']==r, 'blend_50_50'].iloc[0])
                implied.append({
                    'implied_from_rate': r,
                    'AB_next_used': ab_next_pred,
                    'implied_count': r_pred * ab_next_pred
                })

    implied_df = pd.DataFrame(implied)

    meta = {
        'Name': last['Name'],
        'last_year': int(last['year']),
        'age_now': age_now,
        'age_next': age_next,
        'rate_stats_added': rate_stats
    }

    return hist, last_year_df, proj, implied_df, meta

# ---------------------------
# 8) Notebook UI
# ---------------------------
name_box = widgets.Text(
    value="",
    placeholder="Type full or partial name (e.g., 'Mookie Betts')",
    description="Search:",
    layout=widgets.Layout(width='650px')
)
results_dd = widgets.Dropdown(
    options=[],
    description="Matches:",
    layout=widgets.Layout(width='650px')
)
out = widgets.Output()

def refresh_matches(change=None):
    opts = get_matches(name_box.value)
    results_dd.options = opts
    if opts:
        results_dd.value = opts[0][1]

def render_player(change=None):
    with out:
        clear_output(wait=True)

        if results_dd.value is None:
            print("No player selected.")
            return

        hist, last_year_df, proj, implied_df, meta = project_next_year(int(results_dd.value))

        print(meta)

        print("\n--- Last Year Snapshot (includes AB-based rates if available) ---")
        display(last_year_df)

        print("\n--- Full Prior Years (includes AB-based rates if available) ---")
        display(hist)

        print("\n--- Next Year Projections (counts + AB-based rates) ---")
        display(proj)

        if not implied_df.empty:
            print("\n--- Implied counts from predicted AB × predicted rate (blend) ---")
            display(implied_df)

        print("\n--- Model quality by stat (higher R2 is better) ---")
        display(model_metrics)

name_box.observe(refresh_matches, names='value')
results_dd.observe(render_player, names='value')

display(name_box, results_dd, out)
print("✅ Ready. Type a name → pick a match → stats + projections display below.")


Text(value='', description='Search:', layout=Layout(width='650px'), placeholder="Type full or partial name (e.…

Dropdown(description='Matches:', layout=Layout(width='650px'), options=(), value=None)

Output()

✅ Ready. Type a name → pick a match → stats + projections display below.
